# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 12:30PM,261211,19,9387559,"Snap Medical Industries, LLC",Released
1,Apr 17 2023 12:28PM,261176,15,30015671,"Alliance Pharma, Inc.",Released
2,Apr 17 2023 12:28PM,261176,15,30015672,"Alliance Pharma, Inc.",Released
3,Apr 17 2023 12:28PM,261176,15,30015673,"Alliance Pharma, Inc.",Released
4,Apr 17 2023 12:28PM,261176,15,30015675,"Alliance Pharma, Inc.",Released
5,Apr 17 2023 12:28PM,261176,15,30015674,"Alliance Pharma, Inc.",Released
6,Apr 17 2023 12:28PM,261176,15,30015676,"Alliance Pharma, Inc.",Released
7,Apr 17 2023 12:28PM,261176,15,30015677,"Alliance Pharma, Inc.",Released
8,Apr 17 2023 12:28PM,261176,15,30015680,"Alliance Pharma, Inc.",Released
9,Apr 17 2023 12:28PM,261176,15,30015679,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,261206,Released,8
22,261207,Released,1
23,261208,Released,1
24,261209,Released,1
25,261211,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261206,NaN,NaN,8.0
261207,NaN,NaN,1.0
261208,NaN,NaN,1.0
261209,NaN,NaN,1.0
261211,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261145,10.0,2.0,2.0
261146,0.0,0.0,1.0
261147,0.0,1.0,0.0
261161,0.0,0.0,1.0
261168,0.0,25.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261145,10,2,2
261146,0,0,1
261147,0,1,0
261161,0,0,1
261168,0,25,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261145,10,2,2
1,261146,0,0,1
2,261147,0,1,0
3,261161,0,0,1
4,261168,0,25,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261145,10,2,2
1,261146,,,1
2,261147,,1,
3,261161,,,1
4,261168,,25,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 12:30PM,261211,19,"Snap Medical Industries, LLC"
1,Apr 17 2023 12:28PM,261176,15,"Alliance Pharma, Inc."
21,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC"
22,Apr 17 2023 12:09PM,261208,10,Eywa Pharma Inc.
23,Apr 17 2023 12:04PM,261207,19,"GUSA Granules USA, Inc."
24,Apr 17 2023 12:03PM,261206,10,"Snap Medical Industries, LLC"
32,Apr 17 2023 12:01PM,261205,22,"NBTY Global, Inc."
33,Apr 17 2023 11:54AM,261203,10,Emerginnova
34,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.
63,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 17 2023 12:30PM,261211,19,"Snap Medical Industries, LLC",,,1
1,Apr 17 2023 12:28PM,261176,15,"Alliance Pharma, Inc.",,,20
2,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",,,1
3,Apr 17 2023 12:09PM,261208,10,Eywa Pharma Inc.,,,1
4,Apr 17 2023 12:04PM,261207,19,"GUSA Granules USA, Inc.",,,1
5,Apr 17 2023 12:03PM,261206,10,"Snap Medical Industries, LLC",,,8
6,Apr 17 2023 12:01PM,261205,22,"NBTY Global, Inc.",,1,
7,Apr 17 2023 11:54AM,261203,10,Emerginnova,,,1
8,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,,,29
9,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 12:30PM,261211,19,"Snap Medical Industries, LLC",1,,
1,Apr 17 2023 12:28PM,261176,15,"Alliance Pharma, Inc.",20,,
2,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",1,,
3,Apr 17 2023 12:09PM,261208,10,Eywa Pharma Inc.,1,,
4,Apr 17 2023 12:04PM,261207,19,"GUSA Granules USA, Inc.",1,,
5,Apr 17 2023 12:03PM,261206,10,"Snap Medical Industries, LLC",8,,
6,Apr 17 2023 12:01PM,261205,22,"NBTY Global, Inc.",,1,
7,Apr 17 2023 11:54AM,261203,10,Emerginnova,1,,
8,Apr 17 2023 11:48AM,261201,10,Eywa Pharma Inc.,29,,
9,Apr 17 2023 11:25AM,261196,10,"Uniderm USA, Inc",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 12:30PM,261211,19,"Snap Medical Industries, LLC",1,,
1,Apr 17 2023 12:28PM,261176,15,"Alliance Pharma, Inc.",20,,
2,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",1,,
3,Apr 17 2023 12:09PM,261208,10,Eywa Pharma Inc.,1,,
4,Apr 17 2023 12:04PM,261207,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 12:30PM,261211,19,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Apr 17 2023 12:28PM,261176,15,"Alliance Pharma, Inc.",20.0,NaN,NaN
2,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",1.0,NaN,NaN
3,Apr 17 2023 12:09PM,261208,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Apr 17 2023 12:04PM,261207,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 12:30PM,261211,19,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Apr 17 2023 12:28PM,261176,15,"Alliance Pharma, Inc.",20.0,0.0,0.0
2,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",1.0,0.0,0.0
3,Apr 17 2023 12:09PM,261208,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Apr 17 2023 12:04PM,261207,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2350738,98.0,30.0,0.0
12,261209,1.0,0.0,0.0
15,261176,20.0,0.0,0.0
16,261194,1.0,0.0,0.0
18,261161,1.0,0.0,0.0
19,2089400,8.0,3.0,10.0
22,261205,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2350738,98.0,30.0,0.0
1,12,261209,1.0,0.0,0.0
2,15,261176,20.0,0.0,0.0
3,16,261194,1.0,0.0,0.0
4,18,261161,1.0,0.0,0.0
5,19,2089400,8.0,3.0,10.0
6,22,261205,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,98.0,30.0,0.0
1,12,1.0,0.0,0.0
2,15,20.0,0.0,0.0
3,16,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,8.0,3.0,10.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,98.0
1,12,Released,1.0
2,15,Released,20.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,10.0,0.0
Executing,30.0,0.0,0.0,0.0,0.0,3.0,1.0
Released,98.0,1.0,20.0,1.0,1.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,22
0,Completed,0.0,0.0,0.0,0.0,0.0,10.0,0.0
1,Executing,30.0,0.0,0.0,0.0,0.0,3.0,1.0
2,Released,98.0,1.0,20.0,1.0,1.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,22
0,Completed,0.0,0.0,0.0,0.0,0.0,10.0,0.0
1,Executing,30.0,0.0,0.0,0.0,0.0,3.0,1.0
2,Released,98.0,1.0,20.0,1.0,1.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()